In [ ]:
//Load LDA model or the topic distributions
// data format: org.apache.spark.sql.DataFrame = [genome:string, label:string, id:long]
val k_mers_df_train = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/data_train")
val k_mers_df_test = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/data_test")

// data format: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector] the vector part contains the topic distributions
val trainingData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_train_tmp")
val testData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_test_tmp")


  

>     k_mers_df_train: org.apache.spark.sql.DataFrame = [genome: string, label: string ... 1 more field]
>     k_mers_df_test: org.apache.spark.sql.DataFrame = [genome: string, label: string ... 1 more field]
>     trainingData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]
>     testData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]

In [ ]:
testData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_test")


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

firstelement=udf(lambda v:float(v[0]),FloatType())
testData.select(firstelement('_2')).show()

  

>     +------------+
>     |<lambda>(_2)|
>     +------------+
>     | 0.098783955|
>     |  0.09921763|
>     |  0.09954901|
>     |  0.09979748|
>     |  0.09992802|
>     |  0.09876957|
>     |  0.10090187|
>     |  0.10046894|
>     |  0.10041641|
>     | 0.099867895|
>     |  0.10105545|
>     | 0.099601574|
>     |  0.09951745|
>     |  0.10045864|
>     |  0.09878429|
>     |   0.0993379|
>     |  0.10007749|
>     |  0.10007781|
>     |  0.09985662|
>     |   0.1001038|
>     +------------+
>     only showing top 20 rows

  

>     Out[10]: ['<lambda>(_2)', '<lambda>(_2)']

In [ ]:
import org.apache.spark.sql.functions._

val arr_max_df = testData.withColumn("res", array_max($"_2"))

arr_max_df.show()

In [ ]:
val distributions = testData.select("_2")

  

>     distributions: org.apache.spark.sql.DataFrame = [_2: vector]

In [ ]:
distributions.take(2)

  

>     res32: Array[org.apache.spark.sql.Row] = Array([[0.05001287366843784,0.05004600876940525,0.050043300861893174,0.04992445087522449,0.04989027618089037,0.0500213285700815,0.05005898629025325,0.0500518176957626,0.04995232036626174,0.04996466954347265,0.05000450002616852,0.05005636999453604,0.0498470722831295,0.04999132401442033,0.05009426160660837,0.05002498350257868,0.050055494608950715,0.050033006254048176,0.049980093570543414,0.04994686131733314]], [[0.05000556922630469,0.04998299320674995,0.05002190677050916,0.049946103798623666,0.04998885712435416,0.05000591536603424,0.04998201383805509,0.05005154038644628,0.049983023596915956,0.04996312585886514,0.050039226046401285,0.04998483635376038,0.04999263141902274,0.04997888258713219,0.05004847996288107,0.05000765092103313,0.050029108630356776,0.05003844761283082,0.050003191541797484,0.04994649575192555]])

In [ ]:
var rdd2 = distributions.map ( x => x(0))

  

>     rdd2: org.apache.spark.rdd.RDD[Any] = MapPartitionsRDD[379742] at map at command-2294440354339455:1

In [ ]:
rdd2.take(1)

  

>     res49: Array[Any] = Array([0.05001287366843784,0.05004600876940525,0.050043300861893174,0.04992445087522449,0.04989027618089037,0.0500213285700815,0.05005898629025325,0.0500518176957626,0.04995232036626174,0.04996466954347265,0.05000450002616852,0.05005636999453604,0.0498470722831295,0.04999132401442033,0.05009426160660837,0.05002498350257868,0.050055494608950715,0.050033006254048176,0.049980093570543414,0.04994686131733314])

In [ ]:
val newrdd2 = newrdd.map(row => row(0))

In [ ]:
val dfWithSchema = spark.createDataFrame(newrdd).toDF("id")


In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
val mergedTrainingData = trainingData.join(k_mers_df_train,trainingData("_1") === k_mers_df_train("id"),"inner")

  

>     mergedTrainingData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 3 more fields]

In [ ]:
val mergedTestData = testData.join(k_mers_df_test,testData("_1") === k_mers_df_test("id"),"inner")

  

>     mergedTestData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 3 more fields]

In [ ]:
mergedTrainingData.show(5)

  

>     +-----+--------------------+--------------------+------------+-----+
>     |   _1|                  _2|              genome|       label|   id|
>     +-----+--------------------+--------------------+------------+-----+
>     |23928|[0.05218793826923...|ACT CTT TTT TTC T...|northamerica|23928|
>     |23984|[0.05205842680397...|AGA GAT ATC TCT C...|northamerica|23984|
>     |23929|[0.04985890751686...|TAT ATA TAC ACC C...|northamerica|23929|
>     |25242|[0.04823768833064...|TTT TTA TAT ATA T...|northamerica|25242|
>     |23229|[0.05293353532457...|AAG AGG GGT GTT T...|northamerica|23229|
>     +-----+--------------------+--------------------+------------+-----+
>     only showing top 5 rows

In [ ]:
mergedTestData.show(5)

  

>     +----+--------------------+--------------------+-------+----+
>     |  _1|                  _2|              genome|  label|  id|
>     +----+--------------------+--------------------+-------+----+
>     |1277|[0.04996298840239...|ACT CTT TTT TTC T...|oceania|1277|
>     |1224|[0.05000807149599...|ACT CTT TTT TTC T...|oceania|1224|
>     |2173|[0.04997859238978...|AGA GAT ATC TCT C...|oceania|2173|
>     | 541|[0.05000547376345...|CTC TCT CTT TTG T...|oceania| 541|
>     |1371|[0.04997786268672...|AGA GAT ATC TCT C...|oceania|1371|
>     +----+--------------------+--------------------+-------+----+
>     only showing top 5 rows

In [ ]:
mergedTrainingData.schema

  

>     res34: org.apache.spark.sql.types.StructType = StructType(StructField(_1,LongType,true), StructField(_2,org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7,true), StructField(genome,StringType,true), StructField(label,StringType,true), StructField(id,LongType,true))

In [ ]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.lit
import org.apache.commons.math3.distribution.NormalDistribution
val df = (1 to 100).toDF("id")
val r = scala.util.Random
def getOrigin(i: Integer): String=
  if(i<30)
    "Africa"
  else if (i<60)
    "Europe"
  else
    "Asia"
def getDataPoint(i: Integer): Double=
  if(i<30)
    N_africa.sample()
  else if (i<60)
    N_europe.sample()
  else
    N_asia.sample()

var N_europe = new NormalDistribution(2,1)
var N_africa = new NormalDistribution(5,1.4)
var N_asia = new NormalDistribution(3,1.2)
def addColumnsFast(df: DataFrame): DataFrame = 
  df.collect
    .map(_.getInt(0))
    .map(id => (id, getOrigin(id), getDataPoint(id), getDataPoint(id), getDataPoint(id)))
    .toSeq
    .toDF("id", "origin", "topic_1", "topic_2", "topic_3")

val trainingData = spark.time(addColumnsFast(df))
val testData = spark.time(addColumnsFast(df))

trainingData.show(100)

  

>     Time taken: 194 ms
>     Time taken: 35 ms
>     +---+------+------------------+-------------------+--------------------+
>     | id|origin|           topic_1|            topic_2|             topic_3|
>     +---+------+------------------+-------------------+--------------------+
>     |  1|Africa|  5.83777351706712|  7.349972418703414|    6.61662824819416|
>     |  2|Africa|3.8250718188086923|  5.490187621881647|  5.4796719306461785|
>     |  3|Africa|2.5107844489290225|  6.828411793489071|   5.894964244062407|
>     |  4|Africa| 6.751902343774979|  4.715773920336069|   7.950798899603711|
>     |  5|Africa| 5.692182024875181|   5.20516485677467|   6.837812012019507|
>     |  6|Africa| 7.265373268814024|  4.739860849217642|   5.888776980715231|
>     |  7|Africa|3.2850499171125715|  3.014172252632103|    5.64065807813007|
>     |  8|Africa| 6.289192325974664| 3.3346245847805545|  3.8549160253296026|
>     |  9|Africa|3.9449624992374934|  4.484524428773987|   3.786898223532835|
>     | 10|Africa| 4.364116740242096|  3.690682100477985|  5.4829577857011405|
>     | 11|Africa| 4.813145577970281|  7.756058246580167|  6.6158807415724965|
>     | 12|Africa| 4.536488015296846|  6.424314602996467|   3.825316489493102|
>     | 13|Africa| 5.964501998245242|  4.820438789517563|   6.868016673462996|
>     | 14|Africa| 6.988218263075146|  4.653409852364857|   5.247630003939556|
>     | 15|Africa| 4.653698814595546|  4.186782294338912|   6.909289414004915|
>     | 16|Africa| 5.825824728701091| 3.0548104869529946|   5.872787584632095|
>     | 17|Africa| 4.939239964751502|  5.230089812481755|  3.9101615563744705|
>     | 18|Africa|1.9640490923201424|  5.168073776140124|   4.138227945475363|
>     | 19|Africa| 4.054670753427284|  2.856642136219815|   5.874860411326041|
>     | 20|Africa| 7.427811837585756|  5.369465921406486|   4.065292189654439|
>     | 21|Africa| 5.351519156116671|  4.265413371012905|  4.8196777162346995|
>     | 22|Africa|  5.68337982212939|  4.978513534123361|  4.5180667594279305|
>     | 23|Africa|  4.86762722276645|  5.347125964902536|   6.789134685312184|
>     | 24|Africa| 6.712483795813862| 3.6900901582457797|  2.6733882833398286|
>     | 25|Africa| 7.344675173796055| 5.3952987109613035|  3.1632583699352876|
>     | 26|Africa| 4.891463234521708|  2.160898022587189|   4.475577304199971|
>     | 27|Africa| 4.293130640883305|  5.834199502347502|   5.604766756858305|
>     | 28|Africa| 4.211578216587954| 6.8883573685968535|   6.016204384390766|
>     | 29|Africa| 4.611266293274312|  2.933689690450092|   6.651992065621831|
>     | 30|Europe|3.6765421989349236|  0.047400654266885|  2.9576451982000074|
>     | 31|Europe| 2.416951074650528| 1.3220845785226472|  2.3396005595411298|
>     | 32|Europe| 2.201490607308248|  0.695096897340256|  1.4408358991459065|
>     | 33|Europe|2.3671635683815295|  2.682452493450806|  2.9694741564901554|
>     | 34|Europe|1.1607307907197708| 2.2360910479955383|  1.8178768646384096|
>     | 35|Europe| 4.124052516653103|  2.320631725771898|  1.9865335821403907|
>     | 36|Europe|1.5767322333590605| 0.7533220749391005|   3.078069411084817|
>     | 37|Europe|1.6775914480249743|  1.357685719118919|  1.0238736198349492|
>     | 38|Europe| 2.968322357947945|0.21344363600260619|   1.213341252313659|
>     | 39|Europe|3.0165643895827525|   2.83941816961008|  2.9741923335713953|
>     | 40|Europe|1.3985790794418587|  1.786013227033429|  1.5429846540186456|
>     | 41|Europe|3.5622642751626485|  0.602510059206437|  1.9348215936397626|
>     | 42|Europe|1.2053686905376357| 0.6339377129326846|  2.8290277797507253|
>     | 43|Europe|1.5429704074969042| 1.5700372599142745|  2.3973029177901743|
>     | 44|Europe| 1.910235818695376| 1.6703630093901827|  2.7797409480956157|
>     | 45|Europe|1.6176530012553825|0.21168023068729847|   3.213303582037832|
>     | 46|Europe|2.4285004335338134| 2.3687665678543834|  1.9420943878859402|
>     | 47|Europe|2.5324870177742627|   0.52570967355585|   3.423828210873946|
>     | 48|Europe|1.6807051055494195|  2.364873205423158|  1.0938481909831572|
>     | 49|Europe|2.3049433801696217| 0.2235146227952669| 0.43357789441195305|
>     | 50|Europe|0.7543157135804881| 2.0488389767202357|   2.451455697677579|
>     | 51|Europe|0.5307467873488603| 1.9130170826729473|  2.2714855741476407|
>     | 52|Europe| 1.388182975823657| 1.5561333752841193|  0.5924275326329973|
>     | 53|Europe|2.7634338300066323| 3.0960295334308454|  1.6700608572975373|
>     | 54|Europe| 2.228664910039027| 2.0918095484206534|  0.6618346203945285|
>     | 55|Europe|1.6494949542467023|  1.473678414242738|   0.885325678754296|
>     | 56|Europe| 2.797190121751861| 1.1059593492790611|  1.5053048522744459|
>     | 57|Europe| 2.276406175609889|  2.039036200165476|  0.6482979152368846|
>     | 58|Europe|1.6170360969668491|  2.912947294400106|  2.2622734905098056|
>     | 59|Europe|1.9364597680052982| 1.2199270314988566|  1.2293603488526104|
>     | 60|  Asia| 2.100694406775541| 3.2357305267561407|  1.5761706866669734|
>     | 61|  Asia|3.6436191398681634| 2.3691180995941252|  1.4497029445650973|
>     | 62|  Asia|2.6666319929300886|  3.161281243954478|   1.356311476983467|
>     | 63|  Asia| 2.096001393204647|  3.546849940498507|   3.615347257928505|
>     | 64|  Asia| 4.039923533013969|  4.156196310620002|   4.381407357168491|
>     | 65|  Asia|2.4862729998062307|  4.980694561767325|  1.7882130273496375|
>     | 66|  Asia|2.9275997834057645|  5.362816911431885|   5.105952017378549|
>     | 67|  Asia|2.5910943376441367|  4.424325092682374|  1.7032775980219752|
>     | 68|  Asia|3.5627531193489035| 2.9632093769613475|  2.1960928954935843|
>     | 69|  Asia| 4.638136444327561|  6.451716796550906|  3.4187665111355834|
>     | 70|  Asia| 3.174904699322712|  3.791198322755484|  3.0874968772616866|
>     | 71|  Asia|5.5625890094793995|   3.85918227830529|  3.4332912369748234|
>     | 72|  Asia| 2.495982365362969|  3.600982311779159|   1.225450436176291|
>     | 73|  Asia| 2.400928237358435|  4.339591100509332|  1.7292640470886578|
>     | 74|  Asia| 5.762572033481824|  3.861618665894919|  3.5389407588227852|
>     | 75|  Asia| 3.608324898675809| 3.1463801139674104|    2.17813021024272|
>     | 76|  Asia| 2.697825160536849| 1.8921159505446388|   3.429076483513044|
>     | 77|  Asia| 1.989344142250111|  2.383345405429783|   3.660972954683634|
>     | 78|  Asia|1.8819773388968666|  3.646687551781822|  3.3714467236813963|
>     | 79|  Asia|1.7199852927205914| 3.7427485102883393|  1.1839995184491048|
>     | 80|  Asia|2.3325172432677386| 1.5509649044295162|   2.483960254872863|
>     | 81|  Asia|3.3178777405281172|  3.070174219400296|  1.8349741020499355|
>     | 82|  Asia| 4.685885582688538|  4.131419760151927|-0.25254693493518987|
>     | 83|  Asia|3.5806278179087276|  4.536322396781238|  3.9661610988067504|
>     | 84|  Asia| 1.685117816669878|  2.549558489607437|   4.623681261529001|
>     | 85|  Asia| 2.093590641756718|  4.099444775426073|   4.170650095388005|
>     | 86|  Asia|3.0123432402286854| 2.0718636762399982|    4.01309264342162|
>     | 87|  Asia|2.2605922481954748| 3.9550444852898767|  5.0687251737079055|
>     | 88|  Asia| 5.058112895555484| 3.6713441491594643|   3.607109905434706|
>     | 89|  Asia| 3.890010212009255| 2.9569226909295354|  3.8427319732049328|
>     | 90|  Asia|3.0231126938205026|  2.701198685566287|   5.446538264629089|
>     | 91|  Asia| 5.373768022471379| 1.1922653859598078|   4.407538871612573|
>     | 92|  Asia|1.8858250093006175| 0.9939061947899153|  0.9180504458099601|
>     | 93|  Asia|3.1985803797840915| 1.5002982049961184|   4.342711401747792|
>     | 94|  Asia| 4.159178108670287|  3.852581864912148|  1.6810221154876552|
>     | 95|  Asia| 3.083661944024938| 2.7968914458511276|  3.6884215223827694|
>     | 96|  Asia|2.6662339847085303| 1.2744828334224234|  1.0726756282830427|
>     | 97|  Asia|3.3530453490384606| 2.4811202398347265|  2.1610394572609986|
>     | 98|  Asia|2.3440317406475315| 0.5835564496356631|   4.290184707069996|
>     | 99|  Asia|2.9888131321615137| 3.1658196106744976|  3.2779254504841533|
>     |100|  Asia|2.1362574299539587|  4.915812532649392|   4.483981458232519|
>     +---+------+------------------+-------------------+--------------------+
>
>     import org.apache.spark.sql.DataFrame
>     import org.apache.spark.sql.functions.lit
>     import org.apache.commons.math3.distribution.NormalDistribution
>     df: org.apache.spark.sql.DataFrame = [id: int]
>     r: util.Random.type = scala.util.Random$@1172c043
>     getOrigin: (i: Integer)String
>     getDataPoint: (i: Integer)Double
>     N_europe: org.apache.commons.math3.distribution.NormalDistribution = org.apache.commons.math3.distribution.NormalDistribution@6d8dbd8d
>     N_africa: org.apache.commons.math3.distribution.NormalDistribution = org.apache.commons.math3.distribution.NormalDistribution@2a240078
>     N_asia: org.apache.commons.math3.distribution.NormalDistribution = org.apache.commons.math3.distribution.NormalDistribution@3648d171
>     addColumnsFast: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
>     trainingData: org.apache.spark.sql.DataFrame = [id: int, origin: string ... 3 more fields]
>     testData: org.apache.spark.sql.DataFrame = [id: int, origin: string ... 3 more fields]

In [ ]:
//Scatter plots.  Hopefully showing some clustering depending on origin
display(trainingData.sample(false,0.5))

  

[TABLE]

Truncated to 30 rows

In [ ]:
  mergedTrainingData.select("_2").schema

  

>     res31: org.apache.spark.sql.types.StructType = StructType(StructField(_2,org.apache.spark.mllib.linalg.VectorUDT@f71b0bce,true))

In [ ]:
  mergedTrainingData.schema

  

>     res35: org.apache.spark.sql.types.StructType = StructType(StructField(_1,LongType,true), StructField(_2,org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7,true), StructField(genome,StringType,true), StructField(label,StringType,true), StructField(id,LongType,true))

In [ ]:
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.LabeledPoint
import org.apache.spark.ml.classification.RandomForestClassifier

val transformers = Array(
              new StringIndexer().setInputCol("origin").setOutputCol("label"),
              new VectorAssembler()
                     .setInputCols(Array("topic_1","topic_2","topic_3"))
                      .setOutputCol("features"))
val assembler = new VectorAssembler()
    .setInputCols(Array("_2"))
    .setOutputCol("features")
// Train a RandomForest model.
val rf = new RandomForestClassifier() 
              .setLabelCol("label")
              .setFeaturesCol("_2")
              .setNumTrees(10)
              .setFeatureSubsetStrategy("auto")
              .setImpurity("gini")
              .setMaxDepth(20)
              .setMaxBins(32)
              .setSeed(12345)

val tm = assembler.transform(mergedTrainingData)


  

>     import org.apache.spark.ml.feature.{StringIndexer, VectorAssembler}
>     import org.apache.spark.ml.Pipeline
>     import org.apache.spark.ml.feature.LabeledPoint
>     import org.apache.spark.ml.classification.RandomForestClassifier
>     transformers: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.param.shared.HasOutputCol with org.apache.spark.ml.param.shared.HasHandleInvalid with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.param.shared.HasOutputCol with org.apache.spark.ml.param.shared.HasHandleInvalid with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.param.shared.HasOutputCol with org.apache.spark.ml.param.shared.HasHandleInvalid with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(strIdx_424e7f582456, vecAssembler_4f7577042a3a)
>     assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_7d650978c852
>     rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_0a1b095916bc
>     tm: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 4 more fields]

In [ ]:
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.LabeledPoint
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.linalg.Vector

val transformers = Array(
              new StringIndexer().setInputCol("origin").setOutputCol("label"),
             // new VectorAssembler()
             //        .setInputCols(Array("topic_1","topic_2","topic_3"))
             //         .setOutputCol("features"))

// Train a RandomForest model.
val rf = new RandomForestClassifier() 
              .setLabelCol("label")
              .setFeaturesCol("features")
              .setNumTrees(10)
              .setFeatureSubsetStrategy("auto")
              .setImpurity("gini")
              .setMaxDepth(20)
              .setMaxBins(32)
              .setSeed(12345)

val model = new Pipeline().setStages(transformers :+ rf).fit(trainingData)

In [ ]:
val predictionsOnTestData = model.transform(testData)
predictionsOnTestData.show(5)

  

>     predictionsOnTestData: org.apache.spark.sql.DataFrame = [id: int, origin: string ... 8 more fields]

In [ ]:
val predicitonsAndLabels = predictionsOnTestData.select("label", "prediction")

  

>     predicitonsAndLabels: org.apache.spark.sql.DataFrame = [label: double, prediction: double]

In [ ]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictionsOnTestData)
println(s"Test Error = ${(1.0 - accuracy)}")

  

>     Test Error = 0.24
>     import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
>     evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_0fad54a4afcf, metricName=accuracy, metricLabel=0.0, beta=1.0, eps=1.0E-15
>     accuracy: Double = 0.76

In [ ]:
val predictionAndLabelsRdd = predicitonsAndLabels.select("prediction", "label").as[(Double,Double)].rdd

  

>     predictionAndLabelsRdd: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[371802] at rdd at command-2294440354339436:1

In [ ]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val metricsMulti = new MulticlassMetrics(predictionAndLabelsRdd)

  

>     import org.apache.spark.mllib.evaluation.MulticlassMetrics
>     metricsMulti: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@653cdd42

In [ ]:
println("Confusion matrix:")

println(metricsMulti.confusionMatrix)

val accuracy = metricsMulti.accuracy

println("Summary Statistics")

println(s"Accuracy = $accuracy")

  

>     Confusion matrix:
>     28.0  11.0  2.0   
>     7.0   23.0  0.0   
>     4.0   0.0   25.0  
>     Summary Statistics
>     Accuracy = 0.76
>     accuracy: Double = 0.76